# How it works

In this article we go over how `edep2supera` works focusing on 5 points below.
1. Instantiate and configure the module
2. Prepare the input
3. Execute the main functions
4. Interpreting the input
5. Interpreting the output

The first three items show the flow of processes to run `Supera` with `edep-sim` files as input. Then the last two items demonstrate how a user can access the input to and the output from `Supera`.

## Instantiate and Configure

Import `edep2supera` as well as an example module.

In [1]:
import edep2supera
from edep2supera import example

Welcome to JupyROOT 6.22/09


Instantiate and configure `SuperaDriver` for execution of `edep2supera` main functions.

In [2]:
# Instantiate
e2s_driver = edep2supera.edep2supera.SuperaDriver()

# Configure the driver
cfg = example.get_config('edep2supera')
e2s_driver.Configure("edep2supera",cfg)

# Configure the bounding box algorithm
cfg = example.get_config('bbox')
e2s_driver.ConfigureBBoxAlgorithm("BBoxInteraction", cfg)

# Configure the label creation algorithm
cfg = example.get_config('label')
e2s_driver.ConfigureLabelAlgorithm("LArTPCMLReco3D", cfg)

What's going on?
1. `SuperaDriver` is the _driver_ of executing main functions of `edep2supera`
    - A technical detail: it is a wrapper (i.e. C++ inheritance) of `supera::Driver` defined in `SuperaAtomic`.
2. `SuperaDriver` needs three configurations including
    - one for itself (`SuperaDriver.Configure`), and two algorithms:
    - one for creating the meta data of the 3D image volume to be recorded
    - another for interpreting the input and creating necessary information to optimize `lartpc_mlreco3d`
3. Configuration is done by passing a set of parameters. Individual parameter is stored as a key-value pair. For instance, "DeltaSize" (key) and "3" (value). `SuperaDriver` accepts these parameters stored in a container of a type `std::map<string,string>`. 
    - In the above, we use `example` module that can provide example configurations for this tutorial.

We don't go over the full details of the configuration parameters nor the details of the algorithms. For those, refer to the documentation of `SuperaAtomic`. We show how to dump the content for the `SuperaDriver` configuration below.

In [3]:
cfg = example.get_config('edep2supera')

for key,val in cfg:
    print('key: %s\n   - value: %s\n' % (key,val))

key: ActiveDetectors
   - value: [TPCActive_shape]

key: LogLevel
   - value: WARNING



## Reading and Preparing Input

Download an example dataset for this tutorial.

In [4]:
!wget https://web.stanford.edu/~kterao/edep-sim-example.root

--2022-11-14 10:53:11--  https://web.stanford.edu/~kterao/edep-sim-example.root
Resolving web.stanford.edu (web.stanford.edu)... 171.67.215.200, 2607:f6d0:0:925a::ab43:d7c8
Connecting to web.stanford.edu (web.stanford.edu)|171.67.215.200|:443... connected.


HTTP request sent, awaiting response... 

200 OK
Length: 6604072 (6.3M)


Saving to: 'edep-sim-example.root.2'

edep-sim-example.ro   0%[                    ]       0  --.-KB/s               

edep-sim-example.ro   8%[>                   ] 552.00K  2.63MB/s               

edep-sim-example.ro  24%[===>                ]   1.52M  3.74MB/s               

edep-sim-example.ro  40%[=======>            ]   2.52M  4.15MB/s               

edep-sim-example.ro  62%[===========>        ]   3.91M  4.81MB/s               

edep-sim-example.ro  87%[================>   ]   5.51M  5.41MB/s               

edep-sim-example.ro 100%[===================>]   6.30M  5.65MB/s    in 1.1s    

2022-11-14 10:53:13 (5.65 MB/s) - 'edep-sim-example.root.2' saved [6604072/6604072]



This file is generated by (=is an output of) `edep-sim`, which can be easily accessible through `ROOT.TChain`. If there are multiple input files, they can be sequentially added using `TChain.Add` function. To run Supera, we need to:
1. Read one event input from the `edep-sim` output file
2. Interpret `edep-sim` data and generate a formatted input data to run `Supera`
    * This (= being an interface to `edep-sim` file) is exactly why `edep2supera` exists.

In [5]:
# Use TChain to read edep-sim
from ROOT import TChain 
ch=TChain("EDepSimEvents")
ch.AddFile('shower.root')

# Access the first (0-th) input data sample
ch.GetEntry(0)

# Interpret the edep-sim input by handing TG4Event containing all info for this sample.
input_data = e2s_driver.ReadEvent(ch.Event)

AttributeError: 'TChain' object has no attribute 'Event'

Error in <TFile::TFile>: file shower.root does not exist
Error in <TFile::TFile>: file shower.root does not exist
Error in <TFile::TFile>: file shower.root does not exist
Error in <TFile::TFile>: file shower.root does not exist
Error in <TFile::TFile>: file shower.root does not exist


## Running Supera
... is done over two steps. 
* First, `GenerateImageMeta` generates an image meta data.
* Second, `GenerateLabel` generates the main output, infomration necessary for optimizing `lartpc_mlreco3d`.

In [6]:
# Generate image meta data
e2s_driver.GenerateImageMeta(input_data)

# Generate infomration necessary for lartpc_mlreco3d
e2s_driver.GenerateLabel(input_data)

# Access the image meta and output of Supera
meta  = e2s_driver.Meta()
label = e2s_driver.Label()

NameError: name 'input_data' is not defined

## Accessing and Interpreting the Input
Let's take a brief look at the `input_data` which is of `supera::EventInput` type (C++ class defined in `SuperaAtomic`). In short, `input_data` is an array of particle information.

In [7]:
print('Input contains',input_data.size(),'particles')

NameError: name 'input_data' is not defined

Each particle has two essential data elements:
1. Information about this particle
2. A list of energy deposition 3D points with energy deposited and dE/dX at each point

Let's access the former first.

In [8]:
# Print the track ID and PDG code for the first 10 particles stored in the list
for i, p in enumerate(input_data):
    if i == 10: break
    print('Index',i,'Track ID',p.part.trackid,'PDG',p.part.pdg)
    
# Dump the details of the first particle
print('\nDumping information about the first particle...\n')
print(input_data[0].part.dump())

NameError: name 'input_data' is not defined

Note that not all information may be filled at this point, in particular various "ID" (except for Track ID which is aleady determined by the previous simulation step `edep-sim`). 

Next, let's access energy deposition points.

In [9]:
import plotly.graph_objs as go
import numpy as np

particle = input_data[0]
print('There are',particle.pcloud.size(),'points')

# Create a list of points
points = [[pt.x, pt.y, pt.z, pt.e, pt.dedx] for pt in particle.pcloud]
# ... and turn that into a numpy array (so we can slice)
points = np.array(points)

# Create a scatter plot
trace = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2],
                     mode='markers',
                     marker=dict(size=1, color=points[:,3], opacity=0.3),
                    )
fig = go.Figure(data=trace)
fig.show()

NameError: name 'input_data' is not defined

... or plot all points togetehr

In [10]:
points_array = []

for particle in input_data:

    points = [[pt.x, pt.y, pt.z, pt.e, pt.dedx] for pt in particle.pcloud]
    
    if len(points) < 1: continue
    
    points_array.append(np.array(points))

# Concatenate a list of points for all particles
points = np.concatenate(points_array)

print('Total:',len(points),'points',np.sum(points[:,3]),' MeV for',input_data.size(),'particles')

# Create a scatter plot with associated energy in MeV as hovertext
trace = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2],
                     mode='markers',
                     marker=dict(size=1, color=points[:,3], opacity=0.3),
                     hovertext=['%f MeV' % pt[3] for pt in points]                      
                    )
fig = go.Figure(data=trace)
fig.show()

NameError: name 'input_data' is not defined

... or distinguishing each particle by a color? Let's only plot particles with some size.

In [11]:
import plotly
colors = plotly.colors.qualitative.Light24

traces = []
minimum_point_count = 100

for particle in input_data:

    points = [[pt.x, pt.y, pt.z, pt.e, pt.dedx] for pt in particle.pcloud]
    
    if len(points) < minimum_point_count: continue
        
    points = np.array(points)
    color  = colors[particle.part.trackid % len(colors)]
        
    traces.append(go.Scatter3d(x=points[:,0],y=points[:,1],z=points[:,2],
                               mode='markers',
                               name='TID %d PDG %d' % (particle.part.trackid,particle.part.pdg),
                               marker=dict(size=1,color=color,opacity=0.3),
                               hovertext=['EDep %f' % pt[3] for pt in points],
                              )
                 )
    
fig = go.Figure(data=traces)
fig.show()

NameError: name 'input_data' is not defined

## Accessing and Interpreting the Ouput

The output is defined and created by `SuperaAtomic` and there is nothing specific about `edep2supera`. But we can look at a few examples.

In [12]:
# Overall energy deposition is stored in _energies attribute
data = label._energies

# data is VoxelSet type, essentially an array of voxel ID and value = energy.
# We need to take an extra step to translate each voxel ID into XYZ coordinate
xyz = [meta.position(vox.id()) for vox in data.as_vector()]
xyz = [[pt.x, pt.y, pt.z] for pt in xyz]
xyz = np.array(xyz)

energy = [vox.value() for vox in data.as_vector()]
energy = np.array(energy)

trace = go.Scatter3d(x=xyz[:,0],y=xyz[:,1],z=xyz[:,2],
                     mode='markers',
                     marker=dict(size=1,color=energy,opacity=0.3),
                    )
fig = go.Figure(data=trace)
fig.show()

NameError: name 'label' is not defined

Perhaps let's access something unique to the output. An example is a _semantic label_, which is discrete categorization of pixel types. In the below example we ignore pixels of a type `kShapeLEScatter` which represents many small fragments of energy depositions. This allows us to more easily see the trunk of a shower.

In [13]:
from ROOT import supera

# semantic data is stored in an attribute _semanticLabels
# It is of VoxelSet type just like _energies.
data = label._semanticLabels

semantic = [vox.value() for vox in label._semanticLabels.as_vector()]
semantic = np.array(semantic).astype(np.int32)

# Create a pixel mask
mask = np.where(~(semantic==supera.kShapeLEScatter))
# Use the same points (xyz and energy) but apply a mask
trace = go.Scatter3d(x=xyz[mask][:,0],y=xyz[mask][:,1],z=xyz[mask][:,2],
                     mode='markers',
                     marker=dict(size=1,color=energy[mask],opacity=0.3),
                    )
fig = go.Figure(data=trace)
fig.show()

NameError: name 'label' is not defined

We can construct the same view from individual particle information that is also stored in the output. Each particle also stores a semantic type.

In [14]:
traces = []

for particle in label.Particles():
    
    if particle.part.shape == supera.kShapeLEScatter:
        continue
        
    color = colors[particle.part.id % len(colors)]        
    data  = particle.energy
    
    xyz = [meta.position(vox.id()) for vox in data.as_vector()]
    xyz = [[pt.x, pt.y, pt.z] for pt in xyz]
    xyz = np.array(xyz)

    energy = [vox.value() for vox in data.as_vector()]
    energy = np.array(energy)

    traces.append(go.Scatter3d(x=xyz[:,0],y=xyz[:,1],z=xyz[:,2],
                               mode='markers',
                               name='ID %d PDG %d' % (particle.part.id,particle.part.pdg),
                               marker=dict(size=1,color=color,opacity=0.3),
                              )
                 )

fig = go.Figure(data=traces)
fig.show()

NameError: name 'label' is not defined